HPCA(WT) vs. HPCA(N75K)
=======================

In [ ]:
import numpy as np
from numpy import ma
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.animation as anm
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib import colors
from matplotlib.colors import LinearSegmentedColormap

from skimage.util import montage
from skimage.filters import rank
from skimage import morphology
from skimage import exposure
from skimage import measure
from skimage import filters
from skimage import io

from scipy import ndimage
from scipy import signal
from scipy import stats


In [ ]:
from domb.reg_type.wf_x2_m2 import WF_2x_2m as WF
from domb.fret.e_app import Eapp
from domb.red_green.wt_vs_mut import WTvsMut as WvM
from domb.util import plot

In [ ]:
reg_path = 'data/06_06_23/04.tif'
reg_name = reg_path.split('/')[-1]
reg_name = reg_name.split('.')[0]


# fp1 - CFP, fp2 - YFP
reg = WF(img_path=reg_path, img_name=reg_name, ch_order={'fp1':2,'fp2':1}, wf_sigma=.5,
         soma_mask=False, soma_th=.5, soma_ext=150, proc_sigma=.5)  # processes masking parameters

# reg.hist_pic()
reg.ch_pic()
reg.processes_mask_pic()

## RG

In [ ]:
reg_RG = WvM(wt_img=reg.corr_fp1_img,
             mut_img=reg.corr_fp2_img,
             narrow_mask=reg.narrow_proc_mask,
             proc_mask=reg.proc_mask,
             sd_tolerance=2, base_frames=5, app_start=7, app_win=7)  # up mask calc parameters

reg_RG.diff_img_pic()

In [ ]:
plt.imshow(reg_RG.mut_up_label)

## Translocation

In [ ]:
plot.stat_line_plot(arr_list=[reg_RG.wt_init_df_arr,
                              reg_RG.mut_init_df_arr],
                    lab_list=['WT', 'N75K'],
                    stat_method='ci',
                    figsize=(10,4))

In [ ]:
filtered_narrow_mask = morphology.opening(reg.narrow_proc_mask, footprint=morphology.disk(3))

plt.figure(figsize=(15,15))
# plt.imshow(reg_RG.connected_up_mask)
plt.imshow(ma.masked_where(~reg.narrow_proc_mask, reg.narrow_proc_mask), alpha=.5, cmap='bwr')
plt.imshow(ma.masked_where(~filtered_narrow_mask, filtered_narrow_mask), alpha=.5, cmap='bwr')

## FRET

In [ ]:
a = 0.144
b = 0.1292
c = 0.2458
d = 0.8506
abcd = [a, b, c, d]

G = 15.925

reg_FRET = Eapp(dd_img=reg.ch0_img,
                da_img=reg.ch1_img,
                ad_img=reg.ch2_img,
                aa_img=reg.ch3_img,
                abcd_list=abcd, G_val=G,
                mask=reg.proc_mask, corr_by_mask=True)  # parameters for E_corr_calc

In [ ]:
from domb.util.plot import CMaps 

FRET_frame = reg_FRET.R_img[30]

plt.figure(figsize=(10,10))
plt.imshow(ma.masked_where(~reg.proc_mask, FRET_frame),
           cmap='bwr', vmin=-np.max(np.abs(FRET_frame)), vmax=np.max(np.abs(FRET_frame)))

In [ ]:
from domb.util import masking

_,conn_fret = masking.label_prof_arr(input_labels=reg_RG.connected_up_label,
                                   input_img_series=reg_FRET.Ecorr_img)
_,halo_fret = masking.label_prof_arr(input_labels=reg_RG.halo_up_label, 
                                   input_img_series=reg_FRET.Ecorr_img)
_,init_fret = masking.label_prof_arr(input_labels=reg_RG.init_up_label,
                                   input_img_series=reg_FRET.Ecorr_img)

In [ ]:
plot.stat_line_plot(arr_list=[conn_fret,
                              halo_fret,
                              init_fret],
                    lab_list=['conn', 'halo', 'conn'],
                    stat_method='ci',
                    figsize=(10,4))

## dF vs. FRET

In [ ]:
arr_se_stat = lambda x: (np.mean(x, axis=0), \
                         np.std(x, axis=0)/np.sqrt(x.shape[1]))

wt_mean, wt_var = arr_se_stat(reg_RG.wt_halo_df_arr) 
mut_mean, mut_var = arr_se_stat(reg_RG.mut_halo_df_arr)

fret_mean, fret_var = arr_se_stat(halo_fret)


time_line = np.linspace(0, wt_mean.shape[0]*2, \
                            num=wt_mean.shape[0])

fig, ax1 = plt.subplots(figsize=(10,5))

ax1.errorbar(time_line, wt_mean,
             yerr = wt_var,
             fmt ='-o', color='k', capsize=4, label='WT')
ax1.errorbar(time_line, mut_mean,
             yerr = mut_var,
             fmt ='-o', color='r', capsize=4, label='N75K')
ax1.set_xlabel('Time, s')
ax1.set_ylabel('ΔF/F', color='r')
ax1.tick_params(axis ='y', labelcolor='r')
ax1.hlines(y=0, xmin=0, xmax=time_line.max(), linestyles='--', color='k')
ax1.vlines(x=10,
            ymin=wt_mean.min(),
            ymax=wt_mean.max(),
            linestyles=':', color='k', label='Glu application')
plt.legend(loc='upper left')

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis
ax2.errorbar(time_line, fret_mean*100,
             yerr = fret_var*100,
             fmt ='-o', color='b', capsize=4, label='FRET')
ax2.set_ylim([np.mean(np.concatenate((wt_mean, mut_mean))),50])
ax2.set_ylabel('E app, %', color='b')
ax2.tick_params(axis ='y', labelcolor='b')

plt.legend(loc='lower left')
plt.tight_layout()
plt.title('Halo')
plt.show()